In [1]:
%load_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import torch
import gc
import Stemmer
import re
from utils.preprocessing import preprocess_text

### start with 1-hop

In [2]:
metaqa = pd.read_csv("/datasets/MetaQA/1hop/qa_test.txt", sep="\t", header=None)
metaqa.rename(columns={0: "Question", 1: "Answers"}, inplace=True)
metaqa.head()

,Question,Answers
0,what does [Grégoire Colin] appear in,Before the Rain
1,[Joe Thomas] appears in which movies,The Inbetweeners Movie|The Inbetweeners 2
2,what films did [Michelle Trachtenberg] star in,Inspector Gadget|Black Christmas|Ice Princess|...
3,what does [Helen Mack] star in,The Son of Kong|Kiss and Make-Up|Divorce
4,what films did [Shahid Kapoor] act in,Haider|Jab We Met|Chance Pe Dance


In [3]:
mid2name = pd.read_csv("/datasets/MetaQA/KB/kb_entity_dict.txt", sep="\t", header=None)
mid2name.rename(columns={0: "id", 1: "entity"}, inplace=True)
print(len(mid2name))
mid2name.head()

43234


,id,entity
0,0,Kismet
1,1,William Dieterle
2,2,Edward Knoblock
3,3,Marlene Dietrich
4,4,Edward Arnold


In [4]:
metaqa_kb = pd.read_csv("/datasets/MetaQA/KB/kb.txt", sep="|", header=None)
metaqa_kb.rename(columns={0: "subject", 1: "relation", 2: "object"}, inplace=True)
metaqa_kb.head()

,subject,relation,object
0,Kismet,directed_by,William Dieterle
1,Kismet,written_by,Edward Knoblock
2,Kismet,starred_actors,Marlene Dietrich
3,Kismet,starred_actors,Edward Arnold
4,Kismet,starred_actors,Ronald Colman


In [5]:
len(set([*metaqa_kb.subject.unique().tolist(), *metaqa_kb.object.unique().tolist()]).intersection(set(mid2name.entity)))

43234

In [ ]:
from langchain_community.graphs.networkx_graph import KnowledgeTriple
from utils.graph import KGraphPreproc
mqa_graph = KGraphPreproc()
for i, r in metaqa_kb.iterrows():
    triplet = KnowledgeTriple(
        r.subject,
        r.relation,
        r.object,
    )
    mqa_graph.add_triple(triplet)

In [ ]:
mqa_graph.generate_preprocessed_nodes()
# to account for directional edges (llm will figure it out)
mqa_graph._graph = mqa_graph._graph.to_undirected()

### Attempt inference with KG

In [8]:
from langchain.chains import GraphQAChain
from utils.llm.mistral import MistralLLM
from utils.prompt import ENTITY_PROMPT, GRAPH_QA_PROMPT

In [9]:
mistral = MistralLLM()

In [10]:
chain = GraphQAChain.from_llm(
    llm=mistral, 
    graph=mqa_graph,
    qa_prompt=GRAPH_QA_PROMPT,
    entity_prompt=ENTITY_PROMPT,
    verbose=True,
)


In [11]:
q0 = metaqa.iloc[1]
q0.Question

'[Joe Thomas] appears in which movies'

In [13]:
chain.invoke(q0.Question)



> Entering new GraphQAChain chain...
Entities Extracted:
Joe Thomas, movies
Full Context:
Joe Thomas starred_actors The Inbetweeners 2
Joe Thomas starred_actors The Inbetweeners Movie

> Finished chain.


{'query': '[Joe Thomas] appears in which movies',
 'result': 'The Inbetweeners Movie, The Inbetweeners 2'}